## Gene Ontology Term Dataset Creation
### Date: October 07, 2024
### Author: Selin Kaplanoglu

**Goal:** To create a dataset of Gene Ontology (GO) terms under the biological process category to upload on HuggingFace. 


**Information:** This dataset will include three main values of information: GO term and description, the information content, and genes associated with the GO term. The format of this dataset should be a JSON dictionary.

In [ ]:
# install goatools if necessary
# pip install goatools

In [ ]:
import json
import logging
import pandas as pd
import requests
import sys
import time

from goatools.anno.gaf_reader import GafReader
from goatools.obo_parser import GODag
from goatools.semantic import TermCounts, get_info_content

In [ ]:
#information to log into a file
logging.basicConfig(filename='go_terms_log.log', level=logging.INFO, format='%(asctime)s - %(message)s')

In [ ]:
def get_gobp_terms(gobp_annot):
    """
    Extracts the GO BP terms from the GO DAG and GAF data.
    
    Parameters
    ----------
    go_annot: GO BP annotations

    Returns
    -------
    gobp_terms: A set of GO BP terms.
    """
    gobp_terms = set(t for ts in gobp_annot.values() for t in ts)
    return gobp_terms

In [ ]:
def get_gobp_ic(go_dag, gobp_annot, gobp_terms):
    """
    Calculates the information content for each GO BP term.

    Parameters
    ----------
    go_dag: GODag object
    go_annot: GO BP annotations
    gobp_terms: A set of GO BP terms.

    Returns
    -------
    gobp_ic: A dictionary mapping GO terms to their information content.
    """
    #calculate term counts
    gobp_tcs = TermCounts(go_dag, gobp_annot)
    gobp_ic = {k: get_info_content(k, gobp_tcs) for k in gobp_terms}
    return gobp_ic

In [ ]:
def get_go_description(gobp_terms):
    """ Get GO term's name and description from GO API
    ---------
    gobp_terms: A set of GO terms associated with namespace

    Returns
    -------
    go_descriptions: A dictionary with GO term as key and its definition as value
    """
    go_descriptions = {}
    for term in list(gobp_terms):
        requestURL = f"https://api.geneontology.org/api/ontology/term/{term}"
        r = requests.get(requestURL, headers={"Accept": "application/json"})   
        if not r.ok:
            r.raise_for_status()

        response_json = r.json()
        definition = response_json.get('definition')
        go_descriptions[term] = definition

    return go_descriptions

In [ ]:
def create_GOdict(dag_path, gaf_path, out_path):
    """ Creates a JSON list of dictionaries of GO terms
    Parameters
    ----------
    dag_path: Directory path for the Gene Ontology DAG file
    gaf_path: Directory path for the Gene Association file
    out_path: Output file for the JSON dictionary

    Returns
    -------
    json_file: The JSON file of GO term dictionary
    """

    #download the GO DAG
    go_dag = GODag(dag_path)
    
    # download the Gene Association File (GAF)
    go_gaf = GafReader(gaf_path)  

    #subset for BP related GO terms
    gobp_annot = go_gaf.get_id2gos(namespace="BP")
    
    #call get_gobp_terms function to get the GO terms as a set
    gobp_terms = get_gobp_terms(gobp_annot)

    #call get_gobp_ic to compute the information content of each GO term
    gobp_ic = get_gobp_ic(go_dag, gobp_terms, gobp_annot)
    
    #read GAF file and check version
    with open(gaf_path, 'r') as gaf_file:
        header = [next(gaf_file) for _ in range(38)] 
        for line in header:
            if line.startswith("!gaf-version:"):
                gaf_version = line.split(":")[1].strip()
                expected_version = '2.2'
                if gaf_version != expected_version:
                    raise ValueError(f"Expected GAF version {expected_version}, but found version {gaf_version}.")
                break

    # load the GAF file into a DataFrame
    gaf_df = pd.read_csv(gaf_path, skiprows=38, sep="\t", header=None)

    # create a list that will hold all the dictionaries
    go_json = []
    go_descriptions = get_go_description(gobp_terms)

    for term in list(gobp_terms)[:2]:
        start_time = time.time()
        #filters the dataframe for rows where GO terms match the information in the 4th column
        gaf_df_go_terms = gaf_df[gaf_df[4] == term]
        #extract genes from the 2nd column of the filtered dataframe, and store as string
        genes_associated = ','.join(gaf_df_go_terms[2].astype(str))  

        go_json.append({
            'GO_id': term,
            'name': go_dag[term].name,
            'information_content': gobp_ic[term],
            'description': go_descriptions.get(term),
            'genes': genes_associated
        })
        end_time = time.time()
        runtime = end_time - start_time

        #logging it to a log document
        logging.info(f"Time taken for term {term}: {end_time - start_time:.2f} seconds")

    # Write to JSON file
    with open(out_path, 'w') as json_file:
        json.dump(go_json, json_file, indent=4)
    return json_file
    print(f'Successfully created JSON file at {out_path}')



In [ ]:
dag_path = '/mnt/DGX01/Personal/slndir/.go_files/go-basic.obo'
gaf_path = '/mnt/DGX01/Personal/slndir/.go_files/goa_human.gaf'
out_path = '/mnt/DGX01/Personal/slndir/.go_files/go_terms_dict.json'

logging.info("Paths are initialized, starting dictionary creation.")
total_start_time = time.time()

create_GOdict(dag_path, gaf_path, out_path)

total_end_time = time.time()
total_runtime = total_end_time - total_start_time
logging.info(f"Total execution time for all GO terms: {total_runtime:.2f} seconds")
print("Runtime logging complete.")